In [1]:
from Utils.FS import file
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import brown
from scipy.sparse import coo_matrix, dok_matrix
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from wordcloud import WordCloud
from nltk.corpus import stopwords
import tensorflow as tf
import math
import TextPreprocess.words2dict as words2dict

In [2]:
np.random.seed(1234)

In [3]:
sents = brown.sents()
sents = [[token.lower() for token in sent] for sent in sents]
words = brown.words()
words = [word.lower() for word in words]

In [4]:
print("Number of tokens: {}".format(len(words)))
print("Number of sentences: {}".format(len(sents)))
print("Longest sentences length: {}".format(max([len(sent) for sent in sents])))

Number of tokens: 1161192
Number of sentences: 57340
Longest sentences length: 180


In [5]:
words_dict, inv_words_dict = words2dict.convert(words)
words_size = len(words_dict)
print("Number of unique tokens: {}".format(words_size))

Number of unique tokens: 49815


In [6]:
def singleSideWindow(sents, words_dict, window_size, reverse = False):
    window = []
    row = []
    col = []
    data = []
        
    for sent in reversed(sents) if reverse else sents:
        for word in reversed(sent) if reverse else sent:
            for w in window:
                if w == word:
                    continue
                row.append(words_dict[word])
                col.append(words_dict[w])
                data.append(1)
            if len(window) == window_size:
                window.pop(0)
            window.append(word)
    return coo_matrix((data, (row, col)), shape=(len(words_dict), len(words_dict)), dtype='float64')
    

def sents2wordContextMatrix(sents, words_dict, window_size = 5):
    m = coo_matrix((words_size, words_size), 'float64')
    
    print('Doing forward pass...')
    m += singleSideWindow(sents, words_dict, window_size)
    
    print('Doing backward pass...')
    m += singleSideWindow(sents, words_dict, window_size, True)
    
    return m

def sents2wordCoocurrenceMatrix(sents, words_dict, window_size = 10):
    #don't really care edge cases....

    window = []
    row = []
    col = []
    data = []
    for sent in sents:
        for word in sent:
            for i in range(len(window)- 1):
                for j in range(i+1, len(window)):
                    row += [words_dict[window[i]], words_dict[window[j]]]
                    col += [words_dict[window[j]], words_dict[window[i]]]
                    data += [1, 1]
            if len(window) == window_size:
                window.pop(0)
            window.append(word)
    print('Preparing sparse matrix...')
    print('Length of data: {}'.format(len(data)))
    return coo_matrix((data, (row,col)), shape=(words_size, words_size), dtype='float64').multiply(1/len(data))


In [7]:
def singleSideWindowGenerator(c, w, sents, words_dict, window_size = 5, batch_size = 32, reverse = False):
    window = ['--' for i in range(window_size)]
    for sent in reversed(sents) if reverse else sents:
        for word in reversed(sent) if reverse else sent:
            for context in window:
                c.append(words_dict[context])
                w.append([words_dict[word]])
                if(len(c) == batch_size):
                    yield c, w
                    c = []
                    w = []
            window.pop(0)
            window.append(word)

def sents2batchGenerator(sents, words_dict, window_size = 5, batch_size=32):
    c = []
    w = []

    while(True):
        window = ['--' for i in range(window_size)]
        for sent in sents:
            for word in sent:
                for context in window:
                    c.append([words_dict[context]])
                    w.append(words_dict[word])
                    if(len(c) == batch_size):
                        yield w, c
                        c = []
                        w = []
                window.pop(0)
                window.append(word)

        window = ['--' for i in range(window_size)]
        for sent in reversed(sents):
            for word in reversed(sent):
                for context in window:
                    c.append([words_dict[context]])
                    w.append(words_dict[word])
                    if(len(c) == batch_size):
                        yield w, c
                        c = []
                        w = []
                window.pop(0)
                window.append(word)

    

In [8]:
def sents2freq(sents):
    freq = {}
    for sent in sents:
        for word in sent:
            if word in freq.keys():
                freq[word] += 1
            else:
                freq[word] = 1
    return freq

words_freq = sents2freq(sents)

In [9]:
WINDOW_SIZE = 2
BATCH_SIZE = 256
generator = sents2batchGenerator(sents, words_dict, window_size = WINDOW_SIZE, batch_size=BATCH_SIZE)

In [10]:
DIMENSION = 50
VOCABULAY_SIZE = len(words_dict)
NEGATIVE_SAMPLE = 64

graph = tf.Graph()

with graph.as_default():
    
    inputs = tf.placeholder(tf.int32, shape=[BATCH_SIZE])
    labels = tf.placeholder(tf.int32, shape=[BATCH_SIZE, 1])

    with tf.device('/cpu:0'):
    
        embeddings = tf.Variable(
            tf.random_uniform([VOCABULAY_SIZE, DIMENSION], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, inputs, max_norm=1)

        nce_weights = tf.Variable(
            tf.truncated_normal([VOCABULAY_SIZE, DIMENSION],
                                stddev=1.0 / math.sqrt(DIMENSION)))

        nce_biases = tf.Variable(tf.zeros([VOCABULAY_SIZE]))

        loss = tf.reduce_mean(
          tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=labels,
                     inputs=embed,
                     num_sampled=NEGATIVE_SAMPLE,
                     num_classes=VOCABULAY_SIZE))

        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        
        init = tf.global_variables_initializer()
        word2VecSaver = tf.train.Saver({'Words2Vec': embeddings})

In [11]:
num_steps = 200000
MODEL = './model/brown-Words2Vec.ckpt'

with tf.Session(graph=graph) as session:
    init.run()
      
    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = next(generator)
        feed_dict = {inputs: batch_inputs, labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0
    save_path = word2VecSaver.save(session, MODEL)
    final_embeddings = embeddings.eval()

Average loss at step  2000 :  156.574526337
Average loss at step  4000 :  102.405123564
Average loss at step  6000 :  77.9498773155
Average loss at step  8000 :  62.4886512804
Average loss at step  10000 :  51.2490271335
Average loss at step  12000 :  42.7389922895
Average loss at step  14000 :  37.2282794182
Average loss at step  16000 :  31.7435861456
Average loss at step  18000 :  28.2323964698
Average loss at step  20000 :  24.4203342221
Average loss at step  22000 :  21.0401113839
Average loss at step  24000 :  18.4105088477
Average loss at step  26000 :  16.2313612089
Average loss at step  28000 :  14.640471596
Average loss at step  30000 :  13.2159196081
Average loss at step  32000 :  11.9452947013
Average loss at step  34000 :  11.2856972942
Average loss at step  36000 :  10.6026355727
Average loss at step  38000 :  9.97107256126
Average loss at step  40000 :  9.14173763585
Average loss at step  42000 :  8.55057463741
Average loss at step  44000 :  8.04395708036
Average loss at

In [12]:
from sklearn.preprocessing import normalize

normalize(final_embeddings, norm='l2', axis=1, copy=False)
print(final_embeddings.shape)

(49815, 50)


In [13]:
sum((final_embeddings[2] ** 2))

1.000000103886677

In [14]:
words_vec = {}
for i in range(final_embeddings.shape[0]):
    words_vec[inv_words_dict[i]] = final_embeddings[i]

In [15]:
def plotData(vocabs, X, Y):
    plt.clf()
    plt.figure(figsize=(36, 36))
    plt.scatter(X, Y)
    plt.axis([min(X), max(X), min(Y), max(Y)])
    for label, x, y in zip(vocabs, X, Y):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.show()    

In [16]:
def plot(vocabs, words_vec):
    X = [words_vec[vocab][0] for vocab in vocabs]
    Y = [words_vec[vocab][1] for vocab in vocabs]
    plotData(vocabs, X, Y)

In [17]:
def plotTSNE(vocabs, vectors):
    tsne = TSNE(perplexity=30, n_components=2, n_iter=5000, random_state = 7890, method='exact')
    #np.set_printoptions(suppress=True)
    data = np.array([vectors[vocab] for vocab in vocabs])    
    DATA = tsne.fit_transform(data)
    X = DATA[:, 0]
    Y = DATA[:, 1]
    
    plotData(vocabs, X, Y)

In [18]:
from DataLoader import GloVe

glove = GloVe.load2('./data/GloVe/glove.6B.50d.txt')

Start: Loading Glove Model
End: Loaded 400000 rows.


In [19]:
vocabs = ['man', 'woman', 'king', 'queen', 'male', 'female', 'boy', 'girl']
np.random.seed(1234)

random_vocabs = []
for i in np.random.randint(0, len(words_dict), 2000):
    if inv_words_dict[i] in glove.keys():
        random_vocabs.append(inv_words_dict[i])
        
print(len(random_vocabs))

1611


In [20]:
#plotTSNE(random_vocabs, words_vec)

In [21]:
#plotTSNE(random_vocabs, glove)

In [22]:
def cloestWord(word, words_vec, count = 10, method=None):
    if method == 'cos':
        dist = np.array([ sum(words_vec[word] * words_vec[key]) for key in words_vec.keys()])
        top_ten = dist.argsort()[::-1][:10]
    else:
        dist = np.array([ sum(np.square(np.array(words_vec[word]) - np.array(words_vec[key]))) for key in words_vec.keys()])
        top_ten = dist.argsort()[:10]
    return [list(words_vec.keys())[i] for i in top_ten]

In [23]:
print(cloestWord('man', words_vec, method='cos'))
print(cloestWord('man', glove))

['man', 'woman', 'person', 'child', 'boy', 'killed', 'girl', 'married', 'shot', 'further']
['man', 'woman', 'another', 'boy', 'one', 'old', 'turned', 'whose', 'himself', 'who']


In [24]:
print(cloestWord('woman', words_vec, method='cos'))
print(cloestWord('woman', glove))

['woman', 'girl', 'child', 'boy', 'man', 'quiet', 'bad', 'person', 'fine', 'book']
['woman', 'girl', 'man', 'mother', 'boy', 'her', 'she', 'herself', 'victim', 'child']


In [25]:
print(cloestWord('however', words_vec, method='cos'))
print(cloestWord('however', glove))

['however', 'therefore', 'moreover', 'indeed', 'nevertheless', 'jr.', 'especially', 'etc.', 'finally', 'instance']
['however', 'although', 'though', 'as', 'both', 'latter', 'also', '.', 'same', 'fact']


In [26]:
print(cloestWord('his', words_vec, method='cos'))
print(cloestWord('his', glove))

['his', 'their', 'its', 'her', 'our', 'my', 'the', 'every', 'your', 'a']
['his', 'he', 'himself', 'him', 'took', 'came', 'her', 'when', 'having', 'she']


In [27]:
print(cloestWord('zero', words_vec, method='cos'))
print(cloestWord('zero', glove))

['zero', 'prominent', 'touch', 'answers', 'angry', 'beginning', 'train', 'turn', 'orders', 'pictures']
['zero', 'mean', 'assuming', 'hence', 'actual', 'approaching', 'comparable', 'measured', 'limit', 'comparison']


In [28]:
print(cloestWord('one', words_vec, method='cos'))
print(cloestWord('one', glove))

['one', 'importance', 'lack', 'knowledge', 'features', 'cost', 'study', 'meaning', 'value', 'none']
['one', 'another', 'only', 'same', '.', 'as', 'well', 'but', 'with', 'making']


In [29]:
print(cloestWord('two', words_vec, method='cos'))
print(cloestWord('two', glove))

['two', 'three', 'several', 'four', 'other', 'five', 'six', 'ten', 'each', 'types']
['two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'with', 'several']
